In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [10]:
from os import getcwd

In [12]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [7]:
train_df = pd.read_csv("C:/Users/User/train.csv")
test_df = pd.read_csv("C:/Users/User/test.csv")

In [9]:
pd.DataFrame(test_df)

,Image,first_num,second_num
0,0_02.png,0,2
1,100_02.png,0,2
2,101_02.png,0,2
3,102_02.png,0,2
4,103_02.png,0,2
...,...,...,...
19995,997_96.png,9,6
19996,998_96.png,9,6
19997,999_96.png,9,6
19998,99_96.png,9,6


In [11]:
train_images = getcwd() + '/train_images/' + train_df.iloc[:, 0].values
train_images = getcwd() + '/test_images/' + test_df.iloc[:, 0].values

In [ ]:
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [ ]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtypes=tf.float32)
    
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])
    
    labels = {'first num': label[0], 'second num': label[1]}
    
    return image, labels

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
output2 = layers.Dense(10, activation="softmax", name="second_num")(x)

model = keras.Model(inputs=inputs, outputs=[output1, output2])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_dataset, epochs=5, verbose=2)

In [ ]:
model.evaluate(test_dataset, verbose=2)

In [ ]:
# Ссылка на источник - https://www.youtube.com/watch?v=WcZ_1IAH_nM&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=7